In [ ]:
from google.cloud import bigquery
from google.cloud import storage
from datetime import datetime
import logging 
from pycaret.classification import setup,create_model,tune_model, predict_model,get_config,compare_models,save_model,tune_model
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, mean_squared_error, f1_score, precision_score, recall_score, confusion_matrix, roc_curve
import gc
import time
import pandas as pd
import numpy as np
import pickle
from google.cloud import storage
from google.cloud import bigquery
from sklearn.model_selection import train_test_split


In [ ]:
#tag cell with parameters
PROJECT_ID =  'divg-josh-pr-d1cc3a'
BUCKET_NAME='divg-josh-pr-d1cc3a-default'
DATASET_ID = 'telus_rewards'
RESOURCE_BUCKET = 'divg-josh-pr-d1cc3a-default'
FILE_BUCKET = 'divg-josh-pr-d1cc3a-default'
MODEL_ID = '5070'
SERVICE_TYPE = 'telus_rewards'

project_id = PROJECT_ID 
bucket_name = BUCKET_NAME 
dataset_id = DATASET_ID 
resource_bucket = RESOURCE_BUCKET 
file_bucket = FILE_BUCKET 
service_type = SERVICE_TYPE

In [ ]:
import google     
import google.oauth2.credentials
token = !gcloud auth print-access-token
token_str = token[0]

CREDENTIALS = google.oauth2.credentials.Credentials(token)

client = bigquery.Client(project=project_id, location='northamerica-northeast1')
storage_client = storage.Client(project=project_id)


In [ ]:
scoringDate = date(2022, 7, 1)  # date.today() - relativedelta(days=2)- relativedelta(months=30)
valScoringDate = date(2023, 1, 1)  # scoringDate - relativedelta(days=2)

# training dates
SCORE_DATE = scoringDate.strftime('%Y%m%d')  # date.today().strftime('%Y%m%d')
SCORE_DATE_DASH = scoringDate.strftime('%Y-%m-%d')
SCORE_DATE_MINUS_6_MOS_DASH = ((scoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_START_DASH = (scoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_END_DASH = ((scoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
PROMO_EXPIRY_START = (scoringDate.replace(day=1) + relativedelta(months=3)).replace(day=1).strftime('%Y-%m-%d')
PROMO_EXPIRY_END = (scoringDate.replace(day=1) + relativedelta(months=4)).replace(day=1).strftime('%Y-%m-%d')

# validation dates
SCORE_DATE_VAL = valScoringDate.strftime('%Y%m%d')
SCORE_DATE_VAL_DASH = valScoringDate.strftime('%Y-%m-%d')
SCORE_DATE_VAL_MINUS_6_MOS_DASH = ((valScoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_START_DASH = (valScoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_END_DASH = ((valScoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
PROMO_EXPIRY_START_VAL = (valScoringDate.replace(day=1) + relativedelta(months=3)).replace(day=1).strftime('%Y-%m-%d')
PROMO_EXPIRY_END_VAL = (valScoringDate.replace(day=1) + relativedelta(months=4)).replace(day=1).strftime('%Y-%m-%d')

SCORE_DATE_DELTA = 0
SCORE_DATE_VAL_DELTA = 0
TICKET_DATE_WINDOW = 30  # Days of ticket data to be queried


In [ ]:

# josh test starts here
df_train = pd.read_csv('gs://{}/{}_train_monitoring.csv'.format(file_bucket, service_type))

#set up df_train
client = bigquery.Client(project=project_id)
sql_train = ''' SELECT * FROM `{}.{}.bq_telus_rwrd_redemption_targets` '''.format(project_id, dataset_id) 
df_target_train = client.query(sql_train).to_dataframe()
# df_target_train = df_target_train.loc[
#     df_target_train['YEAR_MONTH'] == '-'.join(score_date_dash.split('-')[:2])]  # score_date_dash = '2022-08-31'
df_target_train = df_target_train.loc[df_target_train['YEAR_MONTH'] == '2023']  # score_date_dash = '2022-08-31'
df_target_train['ban'] = df_target_train['ban'].astype('int64')
df_target_train = df_target_train.groupby('ban').tail(1)
df_train = df_train.merge(df_target_train[['ban', 'target_ind']], on='ban', how='left')
df_train.rename(columns={'target_ind': 'target'}, inplace=True)
# df_train.dropna(subset=['target'], inplace=True)
df_train.fillna(0, inplace=True)
df_train['target'] = df_train['target'].astype(int)
print(df_train.shape)

train_all = df_train.copy()

features = [col for col in train_all.columns if col not in ['ban', 'target']]

 ##############  Split train/valid/test based of Dev Training Sample Size   #######################
training_perc = 0.62
train_df = train_all.iloc[:int(train_all.shape[0]*training_perc)]

lower_bound = int(train_all.shape[0]*training_perc)
upper_bound = lower_bound + int(train_all.shape[0]*((1-training_perc)/2))
valid_df = train_all.iloc[lower_bound:upper_bound]

lower_bound = train_df.shape[0] + valid_df.shape[0]
upper_bound = lower_bound + int(train_all.shape[0]*((1-training_perc)/2))
test_df = train_all.iloc[lower_bound:]

drop_cols =  ['ban']

feats = [col for col in train_df.columns if col not in drop_cols]
train_df.drop(columns = drop_cols , inplace=True)
valid_df.drop(columns = drop_cols , inplace=True)

print(train_df.columns)

# train.rpc_flag = train.rpc_flag.astype(int)
train_df.target = train_df.target.astype(int)
valid_df.target = train_df.target.astype(int)
test_df.target = test_df.target.astype(int)

# //todo : Training class 0 and class 1 sample function integration
numeric_features = [col for col in train_df.columns if col != 'target']

################################ Pycaret Setup initialize  ############################ 
classification_setup = setup(data=train_df, 
                         # ignore_features=drop_cols,
                         test_data = valid_df ,
                         target='target',
                         fix_imbalance=False,
                         remove_outliers = True,
                         normalize=True,
                         normalize_method='zscore',
                         log_experiment=False,
                         remove_multicollinearity=True,
                         multicollinearity_threshold=0.95,
                         feature_selection=True,
                         fold=5,
                         fold_shuffle=True,
                         session_id=123,
                         numeric_features=numeric_features,
                         silent=True)

### Pycaret top 3 models to analyze
best_model = compare_models(include = ['rf','xgboost','lightgbm','et'],errors='raise', n_select=3)
# save the model reports and report fig of all top 2 models to GCS
todays_date = datetime.now().strftime("%Y-%m-%d")
save_path = f'pycaret/test_{todays_date}/'
model_reports, model_to_report_map = evaluate_and_save_models(models=best_model.copy(), 
                                     bucket_name=file_bucket,
                                     save_path=save_path, 
                                     test_df=test_df,
                                     actual_label_str='target_ind',
                                     columns = get_config('X_train').columns,
                                     save_columns=True,
                                     show_report=False)

# Find the top Model and top model's report Figs
top_model = None
for i in range(len(best_model)):
    if best_model[i].__class__.__name__ == model_reports.sort_values(["Recall","Precision"],ascending=False).head(1)["model_name"][0]:
        top_model = best_model.copy()[i]


best_model = model_reports.sort_values(["Recall","Precision"],ascending=False).head(1)["model_name"][0]
best_model_report = model_to_report_map[top_model.__class__.__name__]


In [ ]:
true_to_false_ratio = (7013/32853)

def downsampling(df,true_to_false_ratio):
    expected_rpc_flag_0 = df[df["target"]==1]["target"].value_counts().values[0]/true_to_false_ratio
    perc_samples = expected_rpc_flag_0/df[df["target"]==0]["target"].value_counts().values[0]
    
    unique_trips_data = df[df["target"]==0][["ban"]].drop_duplicates()
    unique_trips_sampled = unique_trips_data.sample(int(unique_trips_data.shape[0]*perc_samples))
    
    data_negtv_class = df[df["target"]==0]
    
    data_ngtv_sampled = data_negtv_class[data_negtv_class["ban"].isin(unique_trips_sampled.event_id.values)]
    data_downsampled = pd.concat([data_ngtv_sampled,df[df["target"]!=0]])
    
    return data_downsampled
    

In [ ]:
##############  Downsample train/valid   #######################
sample_perc_training = (7013/32853)
train_sampled = downsampling(df=train_df, true_to_false_ratio=sample_perc_training)

print(train_df.shape)
print(train_sampled.shape)

In [ ]:
train_df.columns